In [98]:
!pip install tensorflow
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
root_dir = "/content/drive/MyDrive/CS 297/mimic-iv-clinical-database-demo-2.2"
admissionsPath = root_dir+"/hosp/admissions.csv.gz"
patientsPath = root_dir + "/hosp/patients.csv.gz"
prescriptionsPath = root_dir + "/hosp/prescriptions.csv.gz"
procedureEventsPath = root_dir + "/icu/procedureevents.csv.gz"
inputEventsPath = root_dir + "/icu/inputevents.csv.gz"
d_itemsPath = root_dir + "/icu/d_items.csv.gz"
chartEventsPath = root_dir + "/icu/chartevents.csv.gz"

SEED = 50

admissionsData = pd.read_csv(admissionsPath)
patientsData = pd.read_csv(patientsPath)
prescriptionsData = pd.read_csv(prescriptionsPath)
procedureEventsData = pd.read_csv(procedureEventsPath)
inputEventsData = pd.read_csv(inputEventsPath)
d_itemsData = pd.read_csv(d_itemsPath)
chartEventsData = pd.read_csv(chartEventsPath)

In [4]:
chartEventsData.head()

,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
0,10005817,20626031,32604416,6770.0,2132-12-16 00:00:00,2132-12-15 23:45:00,225054,On,NaN,NaN,0.0
1,10005817,20626031,32604416,6770.0,2132-12-16 00:00:00,2132-12-15 23:43:00,223769,100,100.0,%,0.0
2,10005817,20626031,32604416,6770.0,2132-12-16 00:00:00,2132-12-15 23:47:00,223956,Atrial demand,NaN,NaN,0.0
3,10005817,20626031,32604416,6770.0,2132-12-16 00:00:00,2132-12-15 23:47:00,224866,Yes,NaN,NaN,0.0
4,10005817,20626031,32604416,6770.0,2132-12-16 00:00:00,2132-12-15 23:45:00,227341,No,0.0,NaN,0.0


In [5]:
chartEventsData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 668862 entries, 0 to 668861
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   subject_id    668862 non-null  int64  
 1   hadm_id       668862 non-null  int64  
 2   stay_id       668862 non-null  int64  
 3   caregiver_id  644622 non-null  float64
 4   charttime     668862 non-null  object 
 5   storetime     667703 non-null  object 
 6   itemid        668862 non-null  int64  
 7   value         668861 non-null  object 
 8   valuenum      257474 non-null  float64
 9   valueuom      162571 non-null  object 
 10  warning       667703 non-null  float64
dtypes: float64(3), int64(4), object(4)
memory usage: 56.1+ MB


In [6]:
d_itemsData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4014 entries, 0 to 4013
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   itemid           4014 non-null   int64  
 1   label            4014 non-null   object 
 2   abbreviation     4014 non-null   object 
 3   linksto          4014 non-null   object 
 4   category         4014 non-null   object 
 5   unitname         1592 non-null   object 
 6   param_type       4014 non-null   object 
 7   lownormalvalue   19 non-null     float64
 8   highnormalvalue  22 non-null     float64
dtypes: float64(2), int64(1), object(6)
memory usage: 282.4+ KB


In [7]:
d_itemsData.head()

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
0,226228,Gender,Gender,chartevents,ADT,NaN,Text,NaN,NaN
1,226545,Race,Race,chartevents,ADT,NaN,Text,NaN,NaN
2,229877,Suction events (CH),Suction events (CH),chartevents,ECMO,NaN,Text,NaN,NaN
3,229875,Oxygenator visible (CH),Oxygenator visible (CH),chartevents,ECMO,NaN,Text,NaN,NaN
4,229266,Cannula sites visually inspected (ECMO),Cannula sites visually inspected (ECMO),chartevents,ECMO,NaN,Text,NaN,NaN


In [8]:
d_itemsData = d_itemsData.drop(columns=['abbreviation', 'linksto', 'unitname', 'param_type', 'lownormalvalue', 'highnormalvalue'])

In [9]:
d_itemsData.head()

,itemid,label,category
0,226228,Gender,ADT
1,226545,Race,ADT
2,229877,Suction events (CH),ECMO
3,229875,Oxygenator visible (CH),ECMO
4,229266,Cannula sites visually inspected (ECMO),ECMO


In [10]:
inputEventsData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20404 entries, 0 to 20403
Data columns (total 26 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   subject_id                     20404 non-null  int64  
 1   hadm_id                        20404 non-null  int64  
 2   stay_id                        20404 non-null  int64  
 3   caregiver_id                   20404 non-null  int64  
 4   starttime                      20404 non-null  object 
 5   endtime                        20404 non-null  object 
 6   storetime                      20404 non-null  object 
 7   itemid                         20404 non-null  int64  
 8   amount                         20404 non-null  float64
 9   amountuom                      20404 non-null  object 
 10  rate                           11038 non-null  float64
 11  rateuom                        11038 non-null  object 
 12  orderid                        20404 non-null 

In [11]:
inputEventsData = inputEventsData[['subject_id', 'hadm_id', 'stay_id', 'starttime', 'endtime', 'storetime', 'itemid', 'amount', 'amountuom', 'rate', 'rateuom', 'ordercategoryname', 'secondaryordercategoryname']]

In [12]:
inputEventsData.head()

,subject_id,hadm_id,stay_id,starttime,endtime,storetime,itemid,amount,amountuom,rate,rateuom,ordercategoryname,secondaryordercategoryname
0,10005817,20626031,32604416,2132-12-16 19:50:00,2132-12-16 19:51:00,2132-12-16 19:50:00,225798,1.0,dose,NaN,NaN,08-Antibiotics (IV),02-Fluids (Crystalloids)
1,10005817,20626031,32604416,2132-12-15 20:15:00,2132-12-15 20:16:00,2132-12-15 20:11:00,225798,1.0,dose,NaN,NaN,08-Antibiotics (IV),02-Fluids (Crystalloids)
2,10005817,20626031,32604416,2132-12-17 09:15:00,2132-12-17 09:16:00,2132-12-17 09:28:00,225798,1.0,dose,NaN,NaN,08-Antibiotics (IV),02-Fluids (Crystalloids)
3,10005817,20626031,32604416,2132-12-16 09:36:00,2132-12-16 09:37:00,2132-12-16 09:37:00,225798,1.0,dose,NaN,NaN,08-Antibiotics (IV),02-Fluids (Crystalloids)
4,10005817,20626031,32604416,2132-12-15 20:10:00,2132-12-15 21:10:00,2132-12-15 20:10:00,221456,2.0,grams,NaN,NaN,02-Fluids (Crystalloids),Additive (Crystalloid)


In [13]:
# Perform the join based on the common field
updatedInputEventsData = pd.merge(d_itemsData, inputEventsData, on=['itemid'], how='inner')

In [14]:
updatedInputEventsData.head()

,itemid,label,category,subject_id,hadm_id,stay_id,starttime,endtime,storetime,amount,amountuom,rate,rateuom,ordercategoryname,secondaryordercategoryname
0,225879,Levofloxacin,Antibiotics,10019777,27738145,34578020,2187-02-13 11:56:00,2187-02-13 11:57:00,2187-02-13 11:56:00,1.0,dose,NaN,NaN,08-Antibiotics (IV),02-Fluids (Crystalloids)
1,225879,Levofloxacin,Antibiotics,10019777,27738145,34578020,2187-02-15 10:03:00,2187-02-15 10:04:00,2187-02-15 10:03:00,1.0,dose,NaN,NaN,08-Antibiotics (IV),02-Fluids (Crystalloids)
2,225879,Levofloxacin,Antibiotics,10019777,27738145,34578020,2187-02-14 10:31:00,2187-02-14 10:32:00,2187-02-14 10:16:00,1.0,dose,NaN,NaN,08-Antibiotics (IV),02-Fluids (Crystalloids)
3,225879,Levofloxacin,Antibiotics,10038933,25129047,32166508,2148-09-14 14:30:00,2148-09-14 14:31:00,2148-09-14 15:24:00,1.0,dose,NaN,NaN,08-Antibiotics (IV),02-Fluids (Crystalloids)
4,225879,Levofloxacin,Antibiotics,10038933,25129047,32166508,2148-09-15 13:23:00,2148-09-15 13:24:00,2148-09-15 13:23:00,1.0,dose,NaN,NaN,08-Antibiotics (IV),02-Fluids (Crystalloids)


In [15]:
chartEventsData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 668862 entries, 0 to 668861
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   subject_id    668862 non-null  int64  
 1   hadm_id       668862 non-null  int64  
 2   stay_id       668862 non-null  int64  
 3   caregiver_id  644622 non-null  float64
 4   charttime     668862 non-null  object 
 5   storetime     667703 non-null  object 
 6   itemid        668862 non-null  int64  
 7   value         668861 non-null  object 
 8   valuenum      257474 non-null  float64
 9   valueuom      162571 non-null  object 
 10  warning       667703 non-null  float64
dtypes: float64(3), int64(4), object(4)
memory usage: 56.1+ MB


In [16]:
chartEventsData = chartEventsData.drop(columns=['caregiver_id'])

In [17]:
chartEventsData.head()

,subject_id,hadm_id,stay_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
0,10005817,20626031,32604416,2132-12-16 00:00:00,2132-12-15 23:45:00,225054,On,NaN,NaN,0.0
1,10005817,20626031,32604416,2132-12-16 00:00:00,2132-12-15 23:43:00,223769,100,100.0,%,0.0
2,10005817,20626031,32604416,2132-12-16 00:00:00,2132-12-15 23:47:00,223956,Atrial demand,NaN,NaN,0.0
3,10005817,20626031,32604416,2132-12-16 00:00:00,2132-12-15 23:47:00,224866,Yes,NaN,NaN,0.0
4,10005817,20626031,32604416,2132-12-16 00:00:00,2132-12-15 23:45:00,227341,No,0.0,NaN,0.0


In [18]:
# Perform the join based on the common field
updatedChartEventsData = pd.merge(d_itemsData, chartEventsData, on=['itemid'], how='inner')

In [19]:
updatedChartEventsData

,itemid,label,category,subject_id,hadm_id,stay_id,charttime,storetime,value,valuenum,valueuom,warning
0,227980,Intra Aortic Balloon Pump Setting,IABP,10002495,24982426,36753294,2141-05-24 01:00:00,2141-05-24 01:07:00,1:1,NaN,NaN,0.0
1,227980,Intra Aortic Balloon Pump Setting,IABP,10002495,24982426,36753294,2141-05-24 02:00:00,2141-05-24 02:49:00,1:2,NaN,NaN,0.0
2,227980,Intra Aortic Balloon Pump Setting,IABP,10002495,24982426,36753294,2141-05-24 03:00:00,2141-05-24 02:49:00,1:2,NaN,NaN,0.0
3,227980,Intra Aortic Balloon Pump Setting,IABP,10002495,24982426,36753294,2141-05-24 04:00:00,2141-05-24 04:25:00,1:1,NaN,NaN,0.0
4,227980,Intra Aortic Balloon Pump Setting,IABP,10002495,24982426,36753294,2141-05-24 05:00:00,2141-05-24 05:56:00,1:1,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
668857,227468,Fibrinogen,Labs,10003400,23559586,34577403,2137-08-10 21:55:00,2137-08-11 00:55:00,247,247.0,mg/dL,0.0
668858,227463,Cortisol,Labs,10039708,28258130,33281088,2140-01-23 19:30:00,2140-01-23 20:50:00,7.4,7.4,ug/dL,0.0
668859,227463,Cortisol,Labs,10039708,28258130,33281088,2140-01-24 05:51:00,2140-01-24 09:31:00,24.6,24.6,ug/dL,1.0
668860,227463,Cortisol,Labs,10020306,23052851,38540883,2135-01-22 02:27:00,2135-01-22 11:04:00,17.8,17.8,ug/dL,0.0


In [20]:
updatedInputEventsData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20404 entries, 0 to 20403
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   itemid                      20404 non-null  int64  
 1   label                       20404 non-null  object 
 2   category                    20404 non-null  object 
 3   subject_id                  20404 non-null  int64  
 4   hadm_id                     20404 non-null  int64  
 5   stay_id                     20404 non-null  int64  
 6   starttime                   20404 non-null  object 
 7   endtime                     20404 non-null  object 
 8   storetime                   20404 non-null  object 
 9   amount                      20404 non-null  float64
 10  amountuom                   20404 non-null  object 
 11  rate                        11038 non-null  float64
 12  rateuom                     11038 non-null  object 
 13  ordercategoryname           204

In [21]:
updatedChartEventsData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 668862 entries, 0 to 668861
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   itemid      668862 non-null  int64  
 1   label       668862 non-null  object 
 2   category    668862 non-null  object 
 3   subject_id  668862 non-null  int64  
 4   hadm_id     668862 non-null  int64  
 5   stay_id     668862 non-null  int64  
 6   charttime   668862 non-null  object 
 7   storetime   667703 non-null  object 
 8   value       668861 non-null  object 
 9   valuenum    257474 non-null  float64
 10  valueuom    162571 non-null  object 
 11  warning     667703 non-null  float64
dtypes: float64(2), int64(4), object(6)
memory usage: 66.3+ MB


In [22]:
updatedInputEventsData2 = updatedInputEventsData.drop(columns=['starttime', 'endtime'])

In [23]:
updatedInputEventsData2['storetime']

0        2187-02-13 11:56:00
1        2187-02-15 10:03:00
2        2187-02-14 10:16:00
3        2148-09-14 15:24:00
4        2148-09-15 13:23:00
                ...         
20399    2125-09-26 05:56:00
20400    2137-08-28 05:26:00
20401    2149-10-03 22:16:00
20402    2149-10-03 14:28:00
20403    2149-10-03 02:20:00
Name: storetime, Length: 20404, dtype: object

In [24]:
updatedInputEventsData2['storetime'] = pd.to_datetime(updatedInputEventsData2['storetime'])
updatedChartEventsData['storetime'] = pd.to_datetime(updatedChartEventsData['storetime'])

In [25]:
updatedInputEventsData2_sorted = updatedInputEventsData2.sort_values(by='storetime')
updatedChartEventsData_sorted = updatedChartEventsData.sort_values(by='storetime')

In [26]:
updatedInputEventsData2_sorted.dropna(subset=['storetime'], inplace=True)
updatedChartEventsData_sorted.dropna(subset=['storetime'], inplace=True)

In [27]:
#data = pd.merge(updatedInputEventsData2_sorted, updatedChartEventsData, on=['subject_id', 'hadm_id', 'stay_id'], how='inner')

In [28]:
# Assuming 'time' is the common time column
merged_data = pd.merge_asof(updatedInputEventsData2_sorted, updatedChartEventsData_sorted, on='storetime')


In [29]:
merged_data

,itemid_x,label_x,category_x,subject_id_x,hadm_id_x,stay_id_x,storetime,amount,amountuom,rate,...,label_y,category_y,subject_id_y,hadm_id_y,stay_id_y,charttime,value,valuenum,valueuom,warning
0,226361,Pre-Admission/Non-ICU Intake,Fluids/Intake,10001725,25563031,31205490,2110-04-11 18:43:00,1000.000000,ml,NaN,...,Teaching directed toward,Adm History/FHPA,10001725,25563031,31205490,2110-04-11 18:40:00,Family Member,NaN,NaN,0.0
1,225158,NaCl 0.9%,Fluids/Intake,10001725,25563031,31205490,2110-04-11 22:51:00,114.666670,ml,10.000000,...,Non Invasive Blood Pressure diastolic,Routine Vital Signs,10001725,25563031,31205490,2110-04-11 21:00:00,74,74.0,mmHg,0.0
2,226452,PO Intake,Fluids/Intake,10001725,25563031,31205490,2110-04-11 22:51:00,120.000000,ml,NaN,...,Non Invasive Blood Pressure diastolic,Routine Vital Signs,10001725,25563031,31205490,2110-04-11 21:00:00,74,74.0,mmHg,0.0
3,229072,Ketorolac (Toradol),Medications,10001725,25563031,31205490,2110-04-11 22:52:00,15.000001,mg,NaN,...,Pain Assessment Method,Pain/Sedation,10001725,25563031,31205490,2110-04-11 22:00:00,Patient Verbalized,NaN,NaN,0.0
4,225944,Sterile Water,Fluids/Intake,10001725,25563031,31205490,2110-04-12 00:34:00,49.999999,ml,25.000000,...,Non Invasive Blood Pressure systolic,Routine Vital Signs,10001725,25563031,31205490,2110-04-12 00:00:00,88,88.0,mmHg,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20399,225975,Heparin Sodium (Prophylaxis),Medications,10027602,21477991,32453351,2201-12-13 12:29:00,1.000000,dose,NaN,...,Alarms On,Alarms,10027602,21477991,32453351,2201-12-13 12:00:00,1,1.0,NaN,0.0
20400,229010,Jevity 1.2 (Full),Nutrition - Enteral,10027602,21477991,32453351,2201-12-13 16:51:00,210.298103,ml,65.040649,...,Pain Present,Pain/Sedation,10027602,21477991,32453351,2201-12-13 16:49:00,Yes,NaN,NaN,0.0
20401,225944,Sterile Water,Fluids/Intake,10027602,21477991,32453351,2201-12-13 16:51:00,49.999999,ml,50.000000,...,Pain Present,Pain/Sedation,10027602,21477991,32453351,2201-12-13 16:49:00,Yes,NaN,NaN,0.0
20402,222011,Magnesium Sulfate,Medications,10027602,21477991,32453351,2201-12-13 16:51:00,2.000000,grams,NaN,...,Pain Present,Pain/Sedation,10027602,21477991,32453351,2201-12-13 16:49:00,Yes,NaN,NaN,0.0


In [30]:
filtered_df = merged_data.query("subject_id_x == subject_id_y and hadm_id_x==hadm_id_y and stay_id_x==stay_id_y")

In [31]:
filtered_df = filtered_df.drop(columns = ['subject_id_x', 'hadm_id_x', 'stay_id_x'])

In [32]:
filtered_df = filtered_df.rename(columns = {'subject_id_y':'subject_id', 'hadm_id_y': 'hadm_id', 'stay_id_y':'stay_id'})

In [33]:
filtered_df

,itemid_x,label_x,category_x,storetime,amount,amountuom,rate,rateuom,ordercategoryname,secondaryordercategoryname,...,label_y,category_y,subject_id,hadm_id,stay_id,charttime,value,valuenum,valueuom,warning
0,226361,Pre-Admission/Non-ICU Intake,Fluids/Intake,2110-04-11 18:43:00,1000.000000,ml,NaN,NaN,16-Pre Admission/Non-ICU,NaN,...,Teaching directed toward,Adm History/FHPA,10001725,25563031,31205490,2110-04-11 18:40:00,Family Member,NaN,NaN,0.0
1,225158,NaCl 0.9%,Fluids/Intake,2110-04-11 22:51:00,114.666670,ml,10.000000,mL/hour,02-Fluids (Crystalloids),Additive (Crystalloid),...,Non Invasive Blood Pressure diastolic,Routine Vital Signs,10001725,25563031,31205490,2110-04-11 21:00:00,74,74.0,mmHg,0.0
2,226452,PO Intake,Fluids/Intake,2110-04-11 22:51:00,120.000000,ml,NaN,NaN,14-Oral/Gastric Intake,NaN,...,Non Invasive Blood Pressure diastolic,Routine Vital Signs,10001725,25563031,31205490,2110-04-11 21:00:00,74,74.0,mmHg,0.0
3,229072,Ketorolac (Toradol),Medications,2110-04-11 22:52:00,15.000001,mg,NaN,NaN,05-Med Bolus,NaN,...,Pain Assessment Method,Pain/Sedation,10001725,25563031,31205490,2110-04-11 22:00:00,Patient Verbalized,NaN,NaN,0.0
4,225944,Sterile Water,Fluids/Intake,2110-04-12 00:34:00,49.999999,ml,25.000000,mL/hour,02-Fluids (Crystalloids),Additive (Crystalloid),...,Non Invasive Blood Pressure systolic,Routine Vital Signs,10001725,25563031,31205490,2110-04-12 00:00:00,88,88.0,mmHg,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20399,225975,Heparin Sodium (Prophylaxis),Medications,2201-12-13 12:29:00,1.000000,dose,NaN,NaN,11-Prophylaxis (Non IV),NaN,...,Alarms On,Alarms,10027602,21477991,32453351,2201-12-13 12:00:00,1,1.0,NaN,0.0
20400,229010,Jevity 1.2 (Full),Nutrition - Enteral,2201-12-13 16:51:00,210.298103,ml,65.040649,mL/hour,13-Enteral Nutrition,Additives (EN),...,Pain Present,Pain/Sedation,10027602,21477991,32453351,2201-12-13 16:49:00,Yes,NaN,NaN,0.0
20401,225944,Sterile Water,Fluids/Intake,2201-12-13 16:51:00,49.999999,ml,50.000000,mL/hour,02-Fluids (Crystalloids),Additive (Crystalloid),...,Pain Present,Pain/Sedation,10027602,21477991,32453351,2201-12-13 16:49:00,Yes,NaN,NaN,0.0
20402,222011,Magnesium Sulfate,Medications,2201-12-13 16:51:00,2.000000,grams,NaN,NaN,02-Fluids (Crystalloids),Additive (Crystalloid),...,Pain Present,Pain/Sedation,10027602,21477991,32453351,2201-12-13 16:49:00,Yes,NaN,NaN,0.0


In [34]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20352 entries, 0 to 20403
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   itemid_x                    20352 non-null  int64         
 1   label_x                     20352 non-null  object        
 2   category_x                  20352 non-null  object        
 3   storetime                   20352 non-null  datetime64[ns]
 4   amount                      20352 non-null  float64       
 5   amountuom                   20352 non-null  object        
 6   rate                        10996 non-null  float64       
 7   rateuom                     10996 non-null  object        
 8   ordercategoryname           20352 non-null  object        
 9   secondaryordercategoryname  14102 non-null  object        
 10  itemid_y                    20352 non-null  int64         
 11  label_y                     20352 non-null  object    

In [35]:
filtered_df.size

427392

In [36]:
filtered_df['itemid_y'].unique

<bound method Series.unique of 0        225131
1        220180
2        220180
3        223795
4        220179
          ...  
20399    224641
20400    223781
20401    223781
20402    223781
20403    223781
Name: itemid_y, Length: 20352, dtype: int64>

In [37]:
from sklearn.preprocessing import OneHotEncoder

In [38]:
def oneHotEncoder(columnName, df, newColumnNames):
  #creating instance of one-hot-encoder
  encoder = OneHotEncoder(handle_unknown='ignore')
  #perform one-hot encoding on 'team' column
  encoder_df = pd.DataFrame(encoder.fit_transform(df[[columnName]]).toarray())
  #print(encoder_df)
  #merge one-hot encoded columns back with original DataFrame
  final_df = df.join(encoder_df)
  final_df = final_df.drop(columns=columnName)
  columnRenaming={}
  for i in range(len(newColumnNames)):
    columnRenaming[i]=newColumnNames[i]
  #print(columnRenaming)
  #final_df = final_df.rename(columns = columnRenaming)
  return final_df

In [39]:
encodedData = oneHotEncoder('itemid_x', filtered_df, 'none')

In [40]:
encodedData

,label_x,category_x,storetime,amount,amountuom,rate,rateuom,ordercategoryname,secondaryordercategoryname,itemid_y,...,149,150,151,152,153,154,155,156,157,158
0,Pre-Admission/Non-ICU Intake,Fluids/Intake,2110-04-11 18:43:00,1000.000000,ml,NaN,NaN,16-Pre Admission/Non-ICU,NaN,225131,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,NaCl 0.9%,Fluids/Intake,2110-04-11 22:51:00,114.666670,ml,10.000000,mL/hour,02-Fluids (Crystalloids),Additive (Crystalloid),220180,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,PO Intake,Fluids/Intake,2110-04-11 22:51:00,120.000000,ml,NaN,NaN,14-Oral/Gastric Intake,NaN,220180,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Ketorolac (Toradol),Medications,2110-04-11 22:52:00,15.000001,mg,NaN,NaN,05-Med Bolus,NaN,223795,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Sterile Water,Fluids/Intake,2110-04-12 00:34:00,49.999999,ml,25.000000,mL/hour,02-Fluids (Crystalloids),Additive (Crystalloid),220179,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20399,Heparin Sodium (Prophylaxis),Medications,2201-12-13 12:29:00,1.000000,dose,NaN,NaN,11-Prophylaxis (Non IV),NaN,224641,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20400,Jevity 1.2 (Full),Nutrition - Enteral,2201-12-13 16:51:00,210.298103,ml,65.040649,mL/hour,13-Enteral Nutrition,Additives (EN),223781,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20401,Sterile Water,Fluids/Intake,2201-12-13 16:51:00,49.999999,ml,50.000000,mL/hour,02-Fluids (Crystalloids),Additive (Crystalloid),223781,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20402,Magnesium Sulfate,Medications,2201-12-13 16:51:00,2.000000,grams,NaN,NaN,02-Fluids (Crystalloids),Additive (Crystalloid),223781,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
grouped_df = filtered_df.groupby('itemid_y')

In [42]:
# Displaying the groups
for group_name, group_data in grouped_df:
    print(f"Group {group_name}:")
    print(group_data['amount'])
    print(group_data['valuenum'])
    print("\n")

Streaming output truncated to the last 5000 lines.
14584      0.571000
14585    120.000000
14586      0.859061
14587     17.843748
19743      1.000000
19744     50.000000
19745     10.000000
19746      0.679000
19747     17.313621
19748    100.000001
19749      1.108072
19750     13.580006
19751     27.259621
19752     28.395431
Name: amount, dtype: float64
4482     1.0
4483     1.0
14583    1.0
14584    1.0
14585    1.0
14586    1.0
14587    1.0
19743    1.0
19744    1.0
19745    1.0
19746    1.0
19747    1.0
19748    1.0
19749    1.0
19750    1.0
19751    1.0
19752    1.0
Name: valuenum, dtype: float64


Group 227429:
15371    349.999987
17966    362.000017
17967    301.666659
Name: amount, dtype: float64
15371    0.01
17966    0.02
17967    0.02
Name: valuenum, dtype: float64


Group 227439:
7636    49.999999
7893     1.000000
7894    50.000000
Name: amount, dtype: float64
7636    0.0
7893    0.0
7894    0.0
Name: valuenum, dtype: float64


Group 227442:
1875     1000.000004
1876   

In [43]:
grouped_df.head()

,itemid_x,label_x,category_x,storetime,amount,amountuom,rate,rateuom,ordercategoryname,secondaryordercategoryname,...,label_y,category_y,subject_id,hadm_id,stay_id,charttime,value,valuenum,valueuom,warning
0,226361,Pre-Admission/Non-ICU Intake,Fluids/Intake,2110-04-11 18:43:00,1000.000000,ml,NaN,NaN,16-Pre Admission/Non-ICU,NaN,...,Teaching directed toward,Adm History/FHPA,10001725,25563031,31205490,2110-04-11 18:40:00,Family Member,NaN,NaN,0.0
1,225158,NaCl 0.9%,Fluids/Intake,2110-04-11 22:51:00,114.666670,ml,10.00000,mL/hour,02-Fluids (Crystalloids),Additive (Crystalloid),...,Non Invasive Blood Pressure diastolic,Routine Vital Signs,10001725,25563031,31205490,2110-04-11 21:00:00,74,74.0,mmHg,0.0
2,226452,PO Intake,Fluids/Intake,2110-04-11 22:51:00,120.000000,ml,NaN,NaN,14-Oral/Gastric Intake,NaN,...,Non Invasive Blood Pressure diastolic,Routine Vital Signs,10001725,25563031,31205490,2110-04-11 21:00:00,74,74.0,mmHg,0.0
3,229072,Ketorolac (Toradol),Medications,2110-04-11 22:52:00,15.000001,mg,NaN,NaN,05-Med Bolus,NaN,...,Pain Assessment Method,Pain/Sedation,10001725,25563031,31205490,2110-04-11 22:00:00,Patient Verbalized,NaN,NaN,0.0
4,225944,Sterile Water,Fluids/Intake,2110-04-12 00:34:00,49.999999,ml,25.00000,mL/hour,02-Fluids (Crystalloids),Additive (Crystalloid),...,Non Invasive Blood Pressure systolic,Routine Vital Signs,10001725,25563031,31205490,2110-04-12 00:00:00,88,88.0,mmHg,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20317,225943,Solution,Fluids/Intake,2201-11-10 08:18:00,50.000000,ml,NaN,NaN,10-Prophylaxis (IV),02-Fluids (Crystalloids),...,Neuro Drain #1 Landmark,Neurological,10027602,28166872,32391858,2201-11-10 08:00:00,Tragus,NaN,NaN,0.0
20318,225907,Famotidine (Pepcid),Medications,2201-11-10 08:18:00,1.000000,dose,NaN,NaN,10-Prophylaxis (IV),02-Fluids (Crystalloids),...,Neuro Drain #1 Landmark,Neurological,10027602,28166872,32391858,2201-11-10 08:00:00,Tragus,NaN,NaN,0.0
20319,225842,Ampicillin,Antibiotics,2201-11-10 08:56:00,1.000000,dose,NaN,NaN,08-Antibiotics (IV),02-Fluids (Crystalloids),...,Neuro Drain #1 Landmark,Neurological,10027602,28166872,32391858,2201-11-10 08:00:00,Tragus,NaN,NaN,0.0
20320,225158,NaCl 0.9%,Fluids/Intake,2201-11-10 08:56:00,1000.000037,ml,75.28231,mL/hour,02-Fluids (Crystalloids),Additive (Crystalloid),...,Neuro Drain #1 Landmark,Neurological,10027602,28166872,32391858,2201-11-10 08:00:00,Tragus,NaN,NaN,0.0


In [44]:
grouped_df

In [50]:
# Extracting and converting only one group (e.g., 'A')
group_a_df = grouped_df.get_group(229247).reset_index(drop=True)

# Now group_a_df is a regular DataFrame containing only the 'A' group
print(group_a_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   itemid_x                    24 non-null     int64         
 1   label_x                     24 non-null     object        
 2   category_x                  24 non-null     object        
 3   storetime                   24 non-null     datetime64[ns]
 4   amount                      24 non-null     float64       
 5   amountuom                   24 non-null     object        
 6   rate                        17 non-null     float64       
 7   rateuom                     17 non-null     object        
 8   ordercategoryname           24 non-null     object        
 9   secondaryordercategoryname  20 non-null     object        
 10  itemid_y                    24 non-null     int64         
 11  label_y                     24 non-null     object        
 

In [91]:
from sklearn.model_selection import train_test_split
def split_dataset(X, y, split_test_size = 0.2, shuffle = False):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split_test_size, random_state=SEED, shuffle=shuffle)

  #Re-indexing
  # X_train = pd.Series(X_train.values.tolist())
  # X_test = pd.Series(X_test.values.tolist())
  # y_train = pd.Series(y_train.values.tolist())
  # y_test = pd.Series(y_test.values.tolist())

  return [X_train, X_test, y_train, y_test]

In [93]:
[X_train, X_test, y_train, y_test] = split_dataset(group_a_df['valuenum'], group_a_df['amount'])

In [53]:
X_train

8      90.0
3      33.0
14     90.0
18    104.0
7      90.0
10     90.0
2      33.0
23    104.0
20    104.0
15     90.0
17     54.0
5      76.0
6      78.0
4      76.0
1      45.0
13     90.0
11     90.0
0      45.0
16     54.0
Name: valuenum, dtype: float64

In [64]:
X_train = np.array(X_train).reshape(-1,1)
X_train.shape

X_test = np.array(X_test).reshape(-1,1)
X_test.shape

(5, 1)

In [65]:
import numpy as np

from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(X_train, np.array(y_train))

print(reg.score(X_test, y_test))
"""
1.0
>>> reg.coef_
array([1., 2.])
>>> reg.intercept_
3.0...
>>> reg.predict(np.array([[3, 5]]))
array([16.])
"""

0.017524915106395422


'\n1.0\n>>> reg.coef_\narray([1., 2.])\n>>> reg.intercept_\n3.0...\n>>> reg.predict(np.array([[3, 5]]))\narray([16.])\n'

In [66]:
!pip install tensorflow

In [75]:
grouped_df_patients = filtered_df.groupby('subject_id')

In [76]:
grouped_df_patients

In [89]:
# Displaying the groups
for group_name, group_data in grouped_df_patients:
    print(f"Group {group_name}:")
    print(group_data['amount'])
    print(group_data['valuenum'])
    print("\n")

Group 10000032:
17882     49.999999
17883     49.999999
17884    100.000000
17885    200.000000
17886    100.000000
Name: amount, dtype: float64
17882     NaN
17883    21.0
17884     NaN
17885     NaN
17886     NaN
Name: valuenum, dtype: float64


Group 10001217:
15396      10.000001
15397     200.000000
15398       1.000000
15399       1.000000
15400     249.999994
15401       1.000000
15402       1.000000
15403     100.000000
15404     200.000000
15405       1.000000
15406    1000.000037
15407      32.500000
15408     360.000000
15409      14.750000
15410     120.000000
15411     120.000000
15412       1.000000
15413       1.000000
15414     250.000000
15415     250.000000
15416       1.000000
15417       1.000000
15418     200.000000
15419       1.000000
15420       2.000000
15421    1000.000000
15422      50.000000
15423       1.000000
15424     308.333326
15425     100.000000
15426       2.000000
15427      10.000001
15428       2.000000
15429       2.000000
15430     200.000000
1

In [90]:
X_train

array([[ 90.],
       [ 33.],
       [ 90.],
       [104.],
       [ 90.],
       [ 90.],
       [ 33.],
       [104.],
       [104.],
       [ 90.],
       [ 54.],
       [ 76.],
       [ 78.],
       [ 76.],
       [ 45.],
       [ 90.],
       [ 90.],
       [ 45.],
       [ 54.]])

In [101]:
#supervised learning model 2: Support Vector Regression
X_train = np.array(X_train).reshape(-1,1)
X_test = np.array(X_test).reshape(-1,1)
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVR

# create an SVR model with a linear kernel
svr = SVR(kernel='linear')

# train the model on the data
svr.fit(X_train, y_train)

# make predictions on the data
y_pred = svr.predict(X_test)

In [73]:
#lstm - can be added to maintain memory of past medical history of patient


# Extracting and converting only one group (e.g., 'A')
group_patient_df = grouped_df_patients.get_group(10039997).reset_index(drop=True)

# Now group_a_df is a regular DataFrame containing only the 'A' group
print(group_patient_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   itemid_x                    35 non-null     int64         
 1   label_x                     35 non-null     object        
 2   category_x                  35 non-null     object        
 3   storetime                   35 non-null     datetime64[ns]
 4   amount                      35 non-null     float64       
 5   amountuom                   35 non-null     object        
 6   rate                        12 non-null     float64       
 7   rateuom                     12 non-null     object        
 8   ordercategoryname           35 non-null     object        
 9   secondaryordercategoryname  19 non-null     object        
 10  itemid_y                    35 non-null     int64         
 11  label_y                     35 non-null     object        
 

In [79]:
[X_train_patient, X_test_patient, y_train_patient, y_test_patient] = split_dataset(group_patient_df['valuenum'], group_patient_df['amount'])

In [81]:
X_train_patient

0      96.2
1       NaN
2       NaN
3       NaN
4       NaN
5       NaN
6       NaN
7       NaN
8       NaN
9       NaN
10      0.0
11      NaN
12      NaN
13      3.0
14      3.0
15      NaN
16      NaN
17     15.0
18    110.0
19    110.0
20    110.0
21      NaN
22      NaN
23     97.2
24     97.2
25    145.0
26    145.0
27    145.0
Name: valuenum, dtype: float64

In [72]:
#rnn
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(units=50, input_shape=(group_patient_df, 20)))
model.add(Dense(units=output_dim))
model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X_train, y_train, epochs=10, batch_size=5)

loss = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")

NameError: ignored

In [71]:
#reinforcement learning implementation